# Product Return Prediction

This project aims to predict whether a purchased product will be returned based on historical sales data.  
The dataset includes features such as category, product version, price reductions, sales tax, purchased item count, and date-related information.  

Data preprocessing steps include handling categorical variables with one-hot encoding and frequency encoding, creating features for weekends and days of the week, and addressing class imbalance using undersampling.  
A Random Forest classifier is used for modeling, and model performance is evaluated using accuracy and confusion matrix.  

The goal is to identify patterns that indicate potential returns to support business decision-making.


## 1. Read data 

In [331]:
## import necessary packages !

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score , confusion_matrix

In [332]:
dataframe = pd.read_csv("order_dataset.csv")

In [333]:
dataframe.head()

,Item Name,Category,Version,Item Code,Item ID,Buyer ID,Transaction ID,Date,Final Quantity,Total Revenue,Price Reductions,Refunds,Final Revenue,Sales Tax,Overall Revenue,Refunded Item Count,Purchased Item Count
0,QID,Product H,32 / B / 30,27-0CD-F44-7E1-0-2F608D7,46567054.0,3301861.0,5.363560e+13,14/04/2019,1,74.17,0.0,0.00,74.17,14.83,89.0,0,1
1,OTH,Product P,32 / B / FtO,37-9D1-AC6-D48-E-F2D4507,16345004.0,1205940.0,4.759180e+13,14/02/2019,-1,0.00,0.0,-79.17,-79.17,-15.83,-95.0,-1,0
2,WHX,Product P,32 / B / FtO,85-2EB-163-D62-5-FC50316,26246865.0,3342830.0,9.211720e+13,28/11/2018,-1,0.00,0.0,-74.17,-74.17,-14.83,-89.0,-1,0
3,RJF,Product P,33 / B / FtO,3D-687-99C-14F-4-661E2E7,42015157.0,7251983.0,5.987730e+13,3/3/2019,1,79.17,0.0,0.00,79.17,15.83,95.0,0,1
4,TSH,Product D,34 / B / FtO,F9-9FA-787-104-B-DCEE379,40522014.0,9940388.0,3.658240e+13,26/11/2018,1,74.17,0.0,0.00,74.17,14.83,89.0,0,1


In [334]:
dataframe.columns ## columns of data

Index(['Item Name', 'Category', 'Version', 'Item Code', 'Item ID', 'Buyer ID',
       'Transaction ID', 'Date', 'Final Quantity', 'Total Revenue',
       'Price Reductions', 'Refunds', 'Final Revenue', 'Sales Tax',
       'Overall Revenue', 'Refunded Item Count', 'Purchased Item Count'],
      dtype='object')

In [335]:
dataframe.shape ## dimensions of data

(70052, 17)

In [336]:
dataframe.isnull() ## Returns False if there is no missing value, True if there is a missing value.

,Item Name,Category,Version,Item Code,Item ID,Buyer ID,Transaction ID,Date,Final Quantity,Total Revenue,Price Reductions,Refunds,Final Revenue,Sales Tax,Overall Revenue,Refunded Item Count,Purchased Item Count
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70047,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
70048,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
70049,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
70050,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


## 2. Do data cleaning/Data preparation !

In [337]:
dataframe.isnull().sum() ## returns total missing values in every column

Item Name               0
Category                0
Version                 0
Item Code               0
Item ID                 0
Buyer ID                0
Transaction ID          0
Date                    0
Final Quantity          0
Total Revenue           0
Price Reductions        0
Refunds                 0
Final Revenue           0
Sales Tax               0
Overall Revenue         0
Refunded Item Count     0
Purchased Item Count    0
dtype: int64

In [338]:
dataframe.dtypes ## tells data-types ofb every feature !

Item Name                object
Category                 object
Version                  object
Item Code                object
Item ID                 float64
Buyer ID                float64
Transaction ID          float64
Date                     object
Final Quantity            int64
Total Revenue           float64
Price Reductions        float64
Refunds                 float64
Final Revenue           float64
Sales Tax               float64
Overall Revenue         float64
Refunded Item Count       int64
Purchased Item Count      int64
dtype: object

Columns that we can use: 
- Category
- Price Reductions
- Total Revenue
- Final Revenue
- Sales Tax
- Purchased Item Count
- Version
- Date

We can use Buyer ID to see total refunds for each customer.

In [339]:
# We can create 3 columns for date.

dataframe['Date'] = pd.to_datetime(dataframe['Date'],dayfirst=True)
dataframe['month'] = dataframe['Date'].dt.month
dataframe['day_of_week'] = dataframe['Date'].dt.dayofweek
dataframe['is_weekend'] = dataframe['day_of_week'].apply(lambda x: 1 if x>=5 else 0)


In [340]:
dataframe.head()

,Item Name,Category,Version,Item Code,Item ID,Buyer ID,Transaction ID,Date,Final Quantity,Total Revenue,Price Reductions,Refunds,Final Revenue,Sales Tax,Overall Revenue,Refunded Item Count,Purchased Item Count,month,day_of_week,is_weekend
0,QID,Product H,32 / B / 30,27-0CD-F44-7E1-0-2F608D7,46567054.0,3301861.0,5.363560e+13,2019-04-14,1,74.17,0.0,0.00,74.17,14.83,89.0,0,1,4,6,1
1,OTH,Product P,32 / B / FtO,37-9D1-AC6-D48-E-F2D4507,16345004.0,1205940.0,4.759180e+13,2019-02-14,-1,0.00,0.0,-79.17,-79.17,-15.83,-95.0,-1,0,2,3,0
2,WHX,Product P,32 / B / FtO,85-2EB-163-D62-5-FC50316,26246865.0,3342830.0,9.211720e+13,2018-11-28,-1,0.00,0.0,-74.17,-74.17,-14.83,-89.0,-1,0,11,2,0
3,RJF,Product P,33 / B / FtO,3D-687-99C-14F-4-661E2E7,42015157.0,7251983.0,5.987730e+13,2019-03-03,1,79.17,0.0,0.00,79.17,15.83,95.0,0,1,3,6,1
4,TSH,Product D,34 / B / FtO,F9-9FA-787-104-B-DCEE379,40522014.0,9940388.0,3.658240e+13,2018-11-26,1,74.17,0.0,0.00,74.17,14.83,89.0,0,1,11,0,0


In [341]:
# We don't need Item Name, Item Code, Item ID, Transaction ID we can drop them.

df = dataframe.drop(['Item Name', 'Item Code', 'Item ID', 'Transaction ID'], axis = 1)

In [342]:
df.head()

,Category,Version,Buyer ID,Date,Final Quantity,Total Revenue,Price Reductions,Refunds,Final Revenue,Sales Tax,Overall Revenue,Refunded Item Count,Purchased Item Count,month,day_of_week,is_weekend
0,Product H,32 / B / 30,3301861.0,2019-04-14,1,74.17,0.0,0.00,74.17,14.83,89.0,0,1,4,6,1
1,Product P,32 / B / FtO,1205940.0,2019-02-14,-1,0.00,0.0,-79.17,-79.17,-15.83,-95.0,-1,0,2,3,0
2,Product P,32 / B / FtO,3342830.0,2018-11-28,-1,0.00,0.0,-74.17,-74.17,-14.83,-89.0,-1,0,11,2,0
3,Product P,33 / B / FtO,7251983.0,2019-03-03,1,79.17,0.0,0.00,79.17,15.83,95.0,0,1,3,6,1
4,Product D,34 / B / FtO,9940388.0,2018-11-26,1,74.17,0.0,0.00,74.17,14.83,89.0,0,1,11,0,0


In [343]:
df['is_returned'] = df.apply(
    lambda row: 1 if (row['Refunded Item Count'] < 0 or row['Final Quantity'] < 0 or row['Refunds'] < 0) else 0, 
    axis=1
)

In [344]:
df['Category'].nunique()

#Number of unique features

23

In [345]:
df['Category']

# The rule of thumb is if your top five or top ten sub categories holds your 70 to 90%of your data, then you can use one hot with some modifications.

0        Product H
1        Product P
2        Product P
3        Product P
4        Product D
           ...    
70047    Product P
70048    Product P
70049    Product D
70050    Product P
70051    Product H
Name: Category, Length: 70052, dtype: object

In [346]:
freq_cat = df['Category'].value_counts()[0:5]/len(df)*100

freq_cat

Product P    33.335237
Product H    22.787358
Product D    10.105350
Product B     5.010564
Product N     4.779307
Name: Category, dtype: float64

In [347]:
freq_cat.index ## these are most frequent values in 'Category' feature..

Index(['Product P', 'Product H', 'Product D', 'Product B', 'Product N'], dtype='object')

In [348]:
df[df['Category'].isin(freq_cat.index)]

,Category,Version,Buyer ID,Date,Final Quantity,Total Revenue,Price Reductions,Refunds,Final Revenue,Sales Tax,Overall Revenue,Refunded Item Count,Purchased Item Count,month,day_of_week,is_weekend,is_returned
0,Product H,32 / B / 30,3301861.0,2019-04-14,1,74.17,0.00,0.00,74.17,14.83,89.0,0,1,4,6,1,0
1,Product P,32 / B / FtO,1205940.0,2019-02-14,-1,0.00,0.00,-79.17,-79.17,-15.83,-95.0,-1,0,2,3,0,1
2,Product P,32 / B / FtO,3342830.0,2018-11-28,-1,0.00,0.00,-74.17,-74.17,-14.83,-89.0,-1,0,11,2,0,1
3,Product P,33 / B / FtO,7251983.0,2019-03-03,1,79.17,0.00,0.00,79.17,15.83,95.0,0,1,3,6,1,0
4,Product D,34 / B / FtO,9940388.0,2018-11-26,1,74.17,0.00,0.00,74.17,14.83,89.0,0,1,11,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70047,Product P,36 / B / FtO,3421398.0,2018-11-20,1,79.16,0.00,0.00,79.16,15.84,95.0,0,1,11,1,0,0
70048,Product P,33 / A / FtO,1913362.0,2019-04-03,1,79.16,-7.91,0.00,71.25,14.25,85.5,0,1,4,2,0,0
70049,Product D,34 / A / FtO,9781863.0,2018-12-26,1,51.67,0.00,0.00,51.67,10.33,62.0,0,1,12,2,0,0
70050,Product P,36 / B / FtO,6808059.0,2019-01-12,1,55.00,0.00,0.00,55.00,11.00,66.0,0,1,1,5,1,0


In [349]:
df['Category'] = df['Category'].apply(lambda x : 'other' if x not in freq_cat.index else x)

## ie, if rest_type doesn't belong to index of freq_cat , we will assign 'other' category to rest_type otherwise remain as it is !

In [350]:
df['Category'].unique()

array(['Product H', 'Product P', 'Product D', 'other', 'Product B',
       'Product N'], dtype=object)

In [351]:
## Applying one-hot on ['Category']
for col in ['Category']:
    col_encoded = pd.get_dummies(df[col] , prefix=col , drop_first=True)
    df = pd.concat([df , col_encoded] , axis=1)

In [352]:
df.head()

,Category,Version,Buyer ID,Date,Final Quantity,Total Revenue,Price Reductions,Refunds,Final Revenue,Sales Tax,...,Purchased Item Count,month,day_of_week,is_weekend,is_returned,Category_Product D,Category_Product H,Category_Product N,Category_Product P,Category_other
0,Product H,32 / B / 30,3301861.0,2019-04-14,1,74.17,0.0,0.00,74.17,14.83,...,1,4,6,1,0,0,1,0,0,0
1,Product P,32 / B / FtO,1205940.0,2019-02-14,-1,0.00,0.0,-79.17,-79.17,-15.83,...,0,2,3,0,1,0,0,0,1,0
2,Product P,32 / B / FtO,3342830.0,2018-11-28,-1,0.00,0.0,-74.17,-74.17,-14.83,...,0,11,2,0,1,0,0,0,1,0
3,Product P,33 / B / FtO,7251983.0,2019-03-03,1,79.17,0.0,0.00,79.17,15.83,...,1,3,6,1,0,0,0,0,1,0
4,Product D,34 / B / FtO,9940388.0,2018-11-26,1,74.17,0.0,0.00,74.17,14.83,...,1,11,0,0,0,1,0,0,0,0


In [353]:
df['Version'].value_counts()/len(df)*100

## One-hot will not be good here as Top 5 or Top 10 categories doesnt have 70-90% of data points

34 / B / FtO    8.131103
32 / A / FtO    6.512305
36 / B / FtO    5.682921
34 / A / FtO    5.434534
32 / B / FtO    5.054816
                  ...   
31 / C / 29     0.001428
28 / C / 31     0.001428
40 / A / 7      0.001428
30 / A / 27     0.001428
40 / B / 28     0.001428
Name: Version, Length: 371, dtype: float64

## 3. Performing Frequency Encoding on Data

Bunu denedikten sonra bir de mean encoding deneyebilirsin. Ama onu train/test ayırmasından sonra yapmalısın, çünkü hedef değişkeni de kullanıyorsun.

In [354]:
df['Version'].nunique()

371

In [355]:
version_freq = df['Version'].value_counts()
df['Version_freq'] = df['Version'].map(version_freq)

In [356]:
df.head()

,Category,Version,Buyer ID,Date,Final Quantity,Total Revenue,Price Reductions,Refunds,Final Revenue,Sales Tax,...,month,day_of_week,is_weekend,is_returned,Category_Product D,Category_Product H,Category_Product N,Category_Product P,Category_other,Version_freq
0,Product H,32 / B / 30,3301861.0,2019-04-14,1,74.17,0.0,0.00,74.17,14.83,...,4,6,1,0,0,1,0,0,0,177
1,Product P,32 / B / FtO,1205940.0,2019-02-14,-1,0.00,0.0,-79.17,-79.17,-15.83,...,2,3,0,1,0,0,0,1,0,3541
2,Product P,32 / B / FtO,3342830.0,2018-11-28,-1,0.00,0.0,-74.17,-74.17,-14.83,...,11,2,0,1,0,0,0,1,0,3541
3,Product P,33 / B / FtO,7251983.0,2019-03-03,1,79.17,0.0,0.00,79.17,15.83,...,3,6,1,0,0,0,0,1,0,3127
4,Product D,34 / B / FtO,9940388.0,2018-11-26,1,74.17,0.0,0.00,74.17,14.83,...,11,0,0,0,1,0,0,0,0,5696


In [357]:
df['is_returned'].value_counts()

0    59269
1    10783
Name: is_returned, dtype: int64

In [358]:
#Drop unnecessary columns
df = df.drop(['Category','Version','Buyer ID','Date','Refunded Item Count','Purchased Item Count','Sales Tax','Refunds','Final Revenue','Total Revenue','Overall Revenue', 'Final Quantity'], axis = 1)

## 4. It's Time For One-Hot-Encoding!
We can add each day to the model as a separate feature by one-hot encoding the day_of_week variable. This allows the model to learn return behavior based on the day of the week.

In [359]:
# One-hot encode day_of_week
day_dummies = pd.get_dummies(df['day_of_week'], prefix='day')

# Append to dataframe
df = pd.concat([df, day_dummies], axis=1)

# Drop the original column
df = df.drop('day_of_week', axis=1)

## 5. Perform Undersampling
In the dataset, there are 59,269 non-returned items and 10,783 returned items, resulting in an imbalance of approximately 5.5–6 times. We can apply undersampling to balance the number of returned items and non-returned items.

In [360]:
# Original X and y
X = df.drop('is_returned', axis=1)
y = df['is_returned']

# Separe returned and non-returned items
df_majority = df[df['is_returned'] == 0]  # non-returned
df_minority = df[df['is_returned'] == 1]  # returned

# Randomly select from the non-returned items the same number of samples as the returned items.
df_majority_downsampled = df_majority.sample(n=len(df_minority), random_state=42)

# New dataframe after undersampling
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# Mix
df_balanced = df_balanced.sample(frac=1, random_state=42)

# Separate as X and y
X_bal = df_balanced.drop('is_returned', axis=1)
y_bal = df_balanced['is_returned']


## 6. Model Building!

In [361]:
X = X_bal ## independent data

In [362]:
y = y_bal ## Dependent data 

In [ ]:
!pip install scikit-learn

In [364]:
## Getting training & testing data!
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify = y)

In [365]:
X_train.shape

(16174, 16)

In [366]:
# Initialize the model with default value of parameters.
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, class_weight='balanced')

In [367]:
# Fit the model to the data.
rf_model.fit(X_train , y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [368]:
predictions = rf_model.predict(X_test)

predictions

array([1, 1, 0, ..., 0, 1, 1], dtype=int64)

In [369]:
accuracy_score(predictions , y_test)

0.7891320474777448

In [370]:
# Compute the error.. 

confusion_matrix(predictions , y_test)

array([[1725,  166],
       [ 971, 2530]], dtype=int64)